In [1]:
from brainlit.utils import upload, session, upload_benchmarking
from pathlib import Path
import napari

c:\users\shrey\anaconda3\envs\ndd-windows\lib\site-packages\python_jsonschema_objects\__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


## Uploading Benchmarking Images from local data locations .
This notebook demonstrates uploading the benchmarking data and associated `.swc` segment files.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `p` is the prefix string. `file://` indicates a filepath, while `s3://` or `gc://` indicate URLs.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, filepaths).
 - `num_res` denotes the number of resolutions to upload. If uploading .tif files that aren't very large or where only one resolution is necessary, set num_res to 1. 
 
The below paths lead to sample data in my local drive (will be updated to reflect download from S3). Alter the below path definitions to point to your own local file locations.

In [2]:
source = Path("C:/Users/shrey/Downloads/1").as_posix()
#source = Path("C:/Users/shrey/Downloads/benchmarking_datasets").as_posix()
source_segments = Path("C:/Users/shrey/Downloads/1/consensus-swcs").as_posix()
dest = "s3://open-neurodata/benchmarking_data/1"
dest_segments = "s3://open-neurodata/benchmarking_data/1"
num_res = 1

## 2) Upload the image data (.tif)

In [3]:
upload_benchmarking.upload_volumes(source, dest, num_res)

Preparing files.
Starting upload.




Finished layer index 0, took 10.890362024307251 seconds


If the upload fails with the error: `timed out on a chunk on layer index 0. moving on to the next step of pipeline`, re-run the `upload_volumes` function but with the `continue_upload` parameter, which takes `layer index` (the layer index said in the error message) and `image index` (the last succesful image that uploaded).  

For example, if the output failed after image 19, then run 
`upload_benchmarking.upload_volumes(source, dest, num_res, continue_upload = (0, 19))`. Repeat this till all of the upload is complete.


## 3) Upload the segmentation data (.swc)

In [3]:
upload_benchmarking.upload_segments(source, dest_segments, num_res)

Uploading:   0%|                                              | 0/1 [00:00<?, ?it/s]

Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 15.19it/s]


In [9]:
img, bounds, vox_in_image = sess.pull_voxel(1, 1)

Downloading: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 27.85it/s]


OutOfBoundsError: Bbox([0, 0, 0],[330, 330, 100], dtype=int32) did not fully contain the specified bounding box Bbox([3864, 896, 542],[3867, 899, 545], dtype=int32).

In [3]:
%%capture
sess = session.NeuroglancerSession(url=dest, url_segments=dest_segments, mip=0)  # create session object object
img, bounds, vertices = sess.pull_vertex_list(1, range(0, 153), expand=False)  # get image containing some data
#labels = sess.create_tubes(1, [330,300,100], radius=1)  # generate labels via tube segmentation

ValueError: operands could not be broadcast together with shapes (0,) (3,) 